In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

import os
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from tensorflow.keras.layers import GaussianNoise
from helpers import *

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.compat.v2.keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from tensorflow.python.client import device_lib

Using TensorFlow backend.


In [3]:
np.random.seed(8)
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3598969880062875668
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3156787200
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3160701173918044119
physical_device_desc: "device: 0, name: GeForce GTX 980M, pci bus id: 0000:01:00.0, compute capability: 5.2"
]


# Loading images

In [4]:
image_dir_train = "data/training/images/"
files = os.listdir(image_dir_train)
n_train = len(files)
print(f"Loading training images, images loaded: {n_train} ")
imgs_train = np.asarray(
    [load_image(image_dir_train + files[i]) for i in range(n_train)]
)
gt_dir_train = "data/training/groundtruth/"
print(f"Loading groundtruth images, images loaded: {n_train} ")
gt_imgs_train = np.asarray(
    [load_image(gt_dir_train + files[i]) for i in range(n_train)]
)

Loading training images, images loaded: 85 
Loading groundtruth images, images loaded: 85 


In [5]:
imgs_train.shape

(85, 400, 400, 3)

In [6]:
gt_imgs_train.shape

(85, 400, 400)

In [7]:
image_size = 400
# Patches for training
img_patches_train = [
    crop_image(imgs_train[i], image_size, image_size) for i in range(n_train)
]
gt_patches_train = [
    crop_image(gt_imgs_train[i], image_size, image_size) for i in range(n_train)
]

# Separate features and labels
X_train = np.asarray(
    [
        img_patches_train[i][j]
        for i in range(len(img_patches_train))
        for j in range(len(img_patches_train[i]))
    ]
)
Y_train = np.asarray(
    [
        gt_patches_train[i][j]
        for i in range(len(gt_patches_train))
        for j in range(len(gt_patches_train[i]))
    ]
)

In [8]:
X_train.shape

(85, 400, 400, 3)

In [9]:
Y_train.shape

(85, 400, 400)

In [10]:
image_dir_val = "data/validating/images/"
files = os.listdir(image_dir_val)
n_val = len(files)
print(f"Loading validating images, images loaded: {n_val} ")
imgs_val = np.asarray([load_image(image_dir_val + files[i]) for i in range(n_val)])
gt_dir_val = "data/validating/groundtruth/"
print(f"Loading validating groundtruth, images loaded: {n_val} ")
gt_imgs_val = np.asarray([load_image(gt_dir_val + files[i]) for i in range(n_val)])

Loading validating images, images loaded: 15 
Loading validating groundtruth, images loaded: 15 


In [11]:
imgs_val.shape

(15, 400, 400, 3)

In [12]:
gt_imgs_val.shape

(15, 400, 400)

In [13]:
image_size = 400
# Patches for validating
img_patches_val = [
    crop_image(imgs_val[i], image_size, image_size) for i in range(n_val)
]
gt_patches_val = [
    crop_image(gt_imgs_val[i], image_size, image_size) for i in range(n_val)
]

# Separate features and labels
X_val = np.asarray(
    [
        img_patches_val[i][j]
        for i in range(len(img_patches_val))
        for j in range(len(img_patches_val[i]))
    ]
)
Y_val = np.asarray(
    [
        gt_patches_val[i][j]
        for i in range(len(gt_patches_val))
        for j in range(len(gt_patches_val[i]))
    ]
)

In [14]:
X_val.shape

(15, 400, 400, 3)

In [15]:
Y_val.shape

(15, 400, 400)

In [16]:
# X_train, Y_train = imag_rotation_aug(imgs_train, gt_imgs_train)

In [17]:
# X_train = np.asarray(X_train)
# Y_train = np.asarray(Y_train)

In [18]:
print(X_train.shape)
print(Y_train.shape)
n_train = Y_train.shape[0]

(85, 400, 400, 3)
(85, 400, 400)


In [19]:
# X_val, Y_val = imag_rotation_aug(imgs_val, gt_imgs_val)

In [20]:
X_val = np.asarray(X_val)
Y_val = np.asarray(Y_val)

In [21]:
print(X_val.shape)
print(Y_val.shape)
n_val = Y_val.shape[0]

(15, 400, 400, 3)
(15, 400, 400)


# Create functions to calcualte precision, recall and F-1 in the training of model

In [22]:
def precision(y_true, y_pred):
    """Compute the Precision for the batch.
    Args:
        y_true (numpy.ndarray): the ground truth labels
        y_pred (numpy.ndarray): the predicted labels 
    Returns:
        Precision (numpy.float64): the Precision of the batch 
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Compute the Recall for the batch.
    Args:
        y_true (numpy.ndarray): the ground truth labels
        y_pred (numpy.ndarray): the predicted labels 
    Returns:
       Recall (numpy.float64): the Recal of the batch 
    """

    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    """Compute the F-1 for the batch.
    Args:
        y_true (numpy.ndarray): the ground truth labels
        y_pred (numpy.ndarray): the predicted labels 
    Returns:
       F-1 (numpy.float64): the F-1 of the batch 
    """
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    return 2 * ((p * r) / (p + r + K.epsilon()))

## ResNet U-Net Architecture

In [23]:
class resnet_unet_model:

    # Initialize the class
    def __init__(self, shape, batch_normalization, activation):
        self.shape = shape
        self.batch_normalization = batch_normalization
        self.activation = activation
        self.model = self.initialize_resnet_unet_model(
            shape, batch_normalization, activation
        )

    def conv_act(self, inputs, out_filters, activation="relu"):
        return Conv2D(
            filters=out_filters,
            activation=activation,
            kernel_size=3,
            strides=1,
            padding="same",
        )(inputs)

    def decoder(
        self,
        inputs,
        mid_filters=512,
        out_filters=256,
        activation="relu",
        block_name="decoder",
    ):
        with K.name_scope(block_name):
            conv = self.conv_act(inputs, mid_filters, activation)
            conv_tr = Conv2DTranspose(
                filters=out_filters,
                activation=activation,
                kernel_size=4,
                strides=2,
                padding="same",
            )(conv)
        return conv_tr

    def initialize_resnet_unet_model(self, shape, batch_normalization, activation):
        #         print(activation)

        # INPUT
        # shape     - Size of the input images
        # OUTPUT
        # model    - Compiled CNN

        # Define parameters
        max_pooling_size = 2
        max_pooling_strd = 2

        # Load a pretrained ResNet
        num_classes = 2
        resnet50 = ResNet50(
            include_top=False,
            weights="imagenet",
            classes=num_classes,
            input_shape=shape,
        )
        resnet50.compile(optimizer=Adam(lr=1e-3), loss="binary_crossentropy")

        # ResNet convolution outputs
        conv5_out = resnet50.get_layer("conv5_block3_out").output
        conv4_out = resnet50.get_layer("conv4_block6_out").output
        conv3_out = resnet50.get_layer("conv3_block4_out").output
        conv2_out = resnet50.get_layer("conv2_block3_out").output

        pool = MaxPooling2D(max_pooling_size, strides=max_pooling_strd, padding="same")(
            resnet50.get_output_at(0)
        )
        dec_center = self.decoder(
            pool,
            mid_filters=shape[0] * 2,
            out_filters=shape[0],
            block_name="decoder_center",
        )
        cat1 = Concatenate()([dec_center, conv5_out])
        dec5 = self.decoder(
            cat1,
            mid_filters=int(shape[0] * 2),
            out_filters=int(shape[0]),
            block_name="decoder5",
        )
        cat2 = Concatenate()([dec5, conv4_out])
        dec4 = self.decoder(
            cat2,
            mid_filters=int(shape[0] * 2),
            out_filters=int(shape[0]),
            block_name="decoder4",
        )
        cat3 = Concatenate()([dec4, conv3_out])
        dec3 = self.decoder(
            cat3,
            mid_filters=int(shape[0]),
            out_filters=int(shape[0] // 4),
            block_name="decoder3",
        )
        cat2 = Concatenate()([dec3, conv2_out])
        dec2 = self.decoder(
            cat2,
            mid_filters=int(shape[0] // 2),
            out_filters=int(shape[0] // 2),
            block_name="decoder2",
        )
        dec1 = self.decoder(
            dec2,
            mid_filters=int(shape[0] // 2),
            out_filters=int(shape[0] // 8),
            block_name="decoder1",
        )
        dec0 = self.conv_act(dec1, out_filters=int(shape[0] // 8))
        conv_f = Conv2D(1, 1, activation="sigmoid", padding="same")(dec0)
        flatten_0 = Flatten()(conv_f)
        dense_0 = Dense(
            shape[0] / 2, kernel_regularizer=l2(1e-6), activity_regularizer=l2(1e-6)
        )(flatten_0)
        lk_relu_0 = LeakyReLU(alpha=0.1)(dense_0)
        dropout_0 = Dropout(0.5)(lk_relu_0)
        dense_1 = Dense(2, kernel_regularizer=l2(1e-6), activity_regularizer=l2(1e-6))(
            dropout_0
        )
        output = Activation("sigmoid")(dense_1)

        model = Model(inputs=resnet50.get_input_at(0), outputs=output)

        # Compile the model using the binary crossentropy loss and the Adam optimizer for it
        # We used the accuracy as a metric, but F1 score is also a plausible choice
        model.compile(
            loss="binary_crossentropy",
            optimizer=Adam(lr=0.001),
            metrics=["accuracy", recall, f1],
        )

        # Print a summary of the model to see what has been generated
        model.summary()

        return model

    def train(self):

        # Early stopping callback after 10 steps
        early_stopping = EarlyStopping(
            monitor="val_loss", min_delta=0, patience=10, verbose=1, mode="auto"
        )

        # Reduce learning rate on plateau after 4 steps
        lr_callback = ReduceLROnPlateau(
            monitor="loss", factor=0.5, patience=4, verbose=1, mode="auto"
        )

        # Save the best model
        weights_filename = "model_"
        if self.batch_normalization:
            weights_filename = weights_filename + "batch_"
        weights_filename = (
            weights_filename
            + self.activation
            + "_"
            + str(EPOCHS)
            + "_"
            + "{epoch:03d}_"
            + "{f1:03f}_"
            + "{val_accuracy:03f}.h5"
        )
        save_best_model = ModelCheckpoint(
            weights_filename,
            save_best_only=True,
            monitor="val_loss",
            mode="auto",
            verbose=1,
        )

        cbs = [save_best_model, lr_callback]

        # Train the model using the previously defined functions and callbacks
        history = self.model.fit_generator(
            create_minibatch(
                X_train,
                Y_train,
                n_train,
                w_size=64,
                batch_size=100,
                patch_size=16,
                width=400,
            ),
            steps_per_epoch=STEPS_PER_EPOCH,
            epochs=EPOCHS,
            use_multiprocessing=False,
            workers=1,
            callbacks=cbs,
            verbose=1,
            validation_data=create_minibatch(
                X_val, Y_val, n_val, w_size=64, batch_size=100, patch_size=16, width=400
            ),
            validation_steps=STEPS_PER_EPOCH,
        )
        
        return history

    def classify(self, X):
        """Classify Image as either road or not.
            Args:
                X (image): part of the image to classify
            Returns:
                Predictions : Predictions for each patch
        """
        # Subdivide the images into blocks with a stride and patch_size of 16
        img_patches = create_patches(X, 16, 16, padding=24)

        # Predict
        predictions = self.model.predict(img_patches)
        predictions = (predictions[:, 0] < predictions[:, 1]) * 1

        # Regroup patches into images
        return predictions.reshape(X.shape[0], -1)

    def load(self, filename):
        """Loads Saved Model.
            Args:
               filename (string): name of the model

        """
        # Load the model (used for submission)
        dependencies = {"recall": recall, "f1": f1}
        self.model = load_model(filename, custom_objects=dependencies)

    def save(self, filename):
        """Saves trained model.
            Args:
               filename (string): name of the model

        """
        self.model.save(filename)

In [27]:
batch_normalization = False
activation = "relu"
amsgrad = False
lr = 1e-4

In [28]:
# Training parameters
EPOCHS = 80
STEPS_PER_EPOCH = 100
batch_size = 100

model = resnet_unet_model(
    shape=(64, 64, 3), batch_normalization=batch_normalization, activation=activation
)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________

# Train the Model


In [29]:
history = model.train()

Epoch 1/80
 99/100 [============================>.] - ETA: 0s - loss: 0.5462 - accuracy: 0.7416 - recall: 0.7165 - f1: 0.7335
Epoch 00001: val_loss improved from inf to 0.96126, saving model to model_relu_80_001_0.734195_0.245700.h5
100/100 [==============================] - 67s 672ms/step - loss: 0.5444 - accuracy: 0.7423 - recall: 0.7172 - f1: 0.7342 - val_loss: 0.9613 - val_accuracy: 0.2457 - val_recall: 0.2457 - val_f1: 0.2457
Epoch 2/80
 99/100 [============================>.] - ETA: 0s - loss: 0.3510 - accuracy: 0.8437 - recall: 0.8389 - f1: 0.8429
Epoch 00002: val_loss did not improve from 0.96126
100/100 [==============================] - 64s 636ms/step - loss: 0.3507 - accuracy: 0.8439 - recall: 0.8392 - f1: 0.8430 - val_loss: 1.7149 - val_accuracy: 0.2491 - val_recall: 0.2491 - val_f1: 0.2491
Epoch 3/80
 99/100 [============================>.] - ETA: 0s - loss: 0.3170 - accuracy: 0.8623 - recall: 0.8580 - f1: 0.8617
Epoch 00003: val_loss did not improve from 0.96126
100/100 [

In [30]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure(figsize=(15, 10))
plt.plot(history.history["loss"][1:], label="train_loss")
plt.plot(history.history["val_loss"][1:], label="val_loss")
plt.plot(history.history["accuracy"][1:], label="train_acc")
plt.plot(history.history["val_accuracy"][1:], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("Unet_batchnorm.png")
plt.show()

AttributeError: 'NoneType' object has no attribute 'history'

<Figure size 1080x720 with 0 Axes>

In [33]:
# Instantiate the model
WINDOW_SIZE = 64
model = resnet_unet_model(
    shape=(WINDOW_SIZE, WINDOW_SIZE, 3),
    batch_normalization=batch_normalization,
    activation=activation,
)

# Load the model
model.load("model_relu_80_028_0.956861_0.912150.h5")

model.model.summary()

# We add all test images to an array, used later for generating a submission
image_filenames = []
for i in range(1, 51):
    image_filename = "data/test_set_images/test_" + str(i) + "/test_" + str(i) + ".png"
    image_filenames.append(image_filename)

# Set-up submission filename
submission_filename = "resnet-unet.csv"

# Generates the submission
generate_submission(model, submission_filename, *image_filenames)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
____________________________________________________________________________________________